In [1]:
import openai
import pandas as pd

In [2]:
client = openai.Client()

In [3]:
file = client.files.create(
    file=open("sales_data.csv", "rb"),
    purpose="assistants"
)

In [4]:
print(file.id)

file-XS73Q1THf6LAP9NQpkipN3


In [5]:
assistant = client.beta.assistants.create(
    name="Analista de Dados",
    instructions="Você é um analista que analisa dados sobre vendas",
    tools=[{"type":"code_interpreter"}],
    tool_resources={"code_interpreter":{"file_ids":[file.id]}},
    model="gpt-4o"
)

In [16]:
# pergunta = "Qual o rating médio das vendas do supermercado"
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto"

In [17]:
# Criação da Thread
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=pergunta
)

In [18]:
# Executa a thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
    instructions="Nome de usuário premium"
)

In [19]:
# Aguarda a thread rodar
import time
while run.status in ["queued", "in_progress", "cancelling"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )

In [10]:
# Verifica a resposta
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print(f"Erro {run.status}")

SyncCursorPage[Message](data=[Message(id='msg_WNuve9rjExRLRfOMtexanxLZ', assistant_id='asst_Y5V4lUZViEavaqmCrWJyrsQN', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O rating médio das vendas do supermercado é aproximadamente 6,94. Se precisar de mais alguma análise ou informação, fique à vontade para perguntar!'), type='text')], created_at=1736970268, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_Em0algAu4MyScr9ZVlXKIaiV', status=None, thread_id='thread_jAUd0kkwLEksHj739BQWQmvZ'), Message(id='msg_h8AC6wKwzhiT6yIwCZOX8HSX', assistant_id='asst_Y5V4lUZViEavaqmCrWJyrsQN', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='O arquivo foi carregado com sucesso e contém dados referentes às transações de um supermercado. Agora que temos acesso aos dados, podemos calcular o rating médio das vendas.\n\nVamos proceder com o cálculo do rating 

In [12]:
mensagens.data[0].content[0].text.value

'O rating médio das vendas do supermercado é aproximadamente 6,94. Se precisar de mais alguma análise ou informação, fique à vontade para perguntar!'

In [20]:
# Analisando os passos do modelo
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id,
    run_id=run.id
)

In [21]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files/{file.id}.png", "wb") as f:
                f.write(image_data.read())
                print(f"Imagem {file_id} salva")
        
        # print(message.content[0].text.value)


====Step message_creation
Vamos começar carregando e inspecionando os dados do arquivo que você enviou. Isso nos permitirá entender sua estrutura e identificar como proceder para gerar o gráfico de pizza.

====Step tool_calls
import pandas as pd

# Carregar o arquivo e inspecionar os primeiros registros
file_path = '/mnt/data/file-XS73Q1THf6LAP9NQpkipN3'
data = pd.read_csv(file_path)

# Exibir as primeiras linhas do dataframe para entender sua estrutura
data.head()

====Step message_creation
Os dados contêm várias colunas, incluindo "Product line" e "Total", que são relevantes para calcular o percentual de vendas por linha de produto. Vamos usar esses dados para criar um gráfico de pizza. Primeiro, precisamos calcular o total de vendas para cada linha de produto e, em seguida, usar esses valores para calcular os percentuais.

Vamos em frente e preparar os dados para o gráfico.

====Step tool_calls
import matplotlib.pyplot as plt

# Calcular o total de vendas por linha de produto
sales